### 4. 얼굴 인식하기

In [1]:
#모듈 임포트
from keras.preprocessing.image import img_to_array
import imutils # OpenCV가 제공하는 기능 중에 좀 복잡하고 사용성이 떨어지는 부분을 잘 보완해 주는 패키지
import cv2 
from keras.models import load_model
import numpy as np

# 모델 가져오기
detection_model_path = 'haarcascade_files/haarcascade_frontalface_default.xml'
#얼굴 감지 모델(haarcascade 트레이닝 데이터를 가져온다.)
emotion_model_path = 'models/_mini_XCEPTION.102-0.66.hdf5'
#표정 감지 모델

# 바운딩 박스 프레임의 하이퍼 파라미터 설정
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
#얼굴 감지 모델을 가져와서 CascadeClassifier 객체를 생성한다.
emotion_classifier = load_model(emotion_model_path, compile=False)
#감정 감지 모델을 가져와서 분류 모델 객체 생성
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]
#데이터셋의 특징


#feelings_faces = []
#for index, emotion in enumerate(EMOTIONS):
   # feelings_faces.append(cv2.imread('emojis/' + emotion + '.png', -1))

# starting video streaming
cv2.namedWindow('your_face') #창 이름
camera = cv2.VideoCapture(0) #비디오 ON! 0 = 첫번째 인덱스 카메라
while True: #무한 반복
    frame = camera.read()[1] #카메라를 읽는다.
    #카메라 창 읽기
    frame = imutils.resize(frame,width=300) #카메라(frame)사이즈 조정
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #cvtColor 함수를 이용하여 기본 BGR컬러를 Gray로 바꾼다.
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    #detectMultiScale에 grayscale 이미지를 입력하여 얼굴을 검출한다. 검출되면 위치로 리스트한다. 링크참조.
    canvas = np.zeros((250, 300, 3), dtype="uint8") #canvas 사이즈
    frameClone = frame.copy() #FrameClone <- Frame (배열복제)
    if len(faces) > 0: #얼굴 검출 설정 (얼굴 영역)
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
                    # 그레이스케일 영상에서 얼굴의 roi를 추출하고 고정된 28x28 픽셀로 크기를 조정한 다음 준비한다.
            # CNN을 통한 roi 분류
        roi = gray[fY:fY + fH, fX:fX + fW] #roi(관심영역 지정)
        roi = cv2.resize(roi, (64, 64)) # 관심영역 사이즈
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0) 
        
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
    else: continue

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # 라벨의 문자 설정
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # cavas 창에 라벨과 확률치 바를 만든다.
               # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5), #확률치 바
                (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23), #텍스트
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, #폰트 설정
                (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10), #카메라에 텍스트 관심영역 -10 (위로 10)
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2) #폰트 설정
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2) #카메라의 프레임(관심영역)
#    for c in range(0, 3):
#        frame[200:320, 10:130, c] = emoji_face[:, :, c] * \
#        (emoji_face[:, :, 3] / 255.0) + frame[200:320,
#        10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)


    cv2.imshow('your_face', frameClone) #카메라창(frameClone)을 출력한다.
    cv2.imshow("Probabilities", canvas) #특성창(canvas)를 출력한다..
    if cv2.waitKey(1) & 0xFF == ord('q'): #Q를 누르면 나간다.
        break

camera.release() #카메라를 닫는다.
cv2.destroyAllWindows() #윈도우 창을 끈다.


Using TensorFlow backend.


KeyboardInterrupt: 